# Aula 1 - Criando um agente de funções

## Vídeo 1.2 - Configurando o LlamaIndex

In [1]:
!pip install llama-index llama_index.embeddings.huggingface llama-index-readers-file llama-index-llms-groq arxiv llama-index-tools-tavily-research crewai crewai-tools gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of lancedb to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━

In [2]:
from llama_index.core import Settings

In [3]:
from google.colab import userdata
groq = userdata.get('GROQ_API_KEY')

In [4]:
from llama_index.llms.groq import Groq

llm = Groq(model="llama-3.3-70b-versatile", api_key=groq)

In [5]:
Settings.llm = llm

In [6]:
def calcular_engajamento(curtidas: int, comentarios: int, compartilhamentos: int, seguidores: int) -> str:
    """
    Calcula o engajamento total e a taxa de engajamento de uma postagem.

    Args:
        curtidas (int): Número de curtidas na postagem.
        comentarios (int): Número de comentários na postagem.
        compartilhamentos (int): Número de compartilhamentos da postagem.
        seguidores (int): Número total de seguidores.

    Returns:
        dict: Um dicionário contendo o engajamento total (int) e a taxa de engajamento (float, em porcentagem).
    """
    engajamento_total = curtidas + comentarios + compartilhamentos
    taxa_engajamento = (engajamento_total / seguidores) * 100 if seguidores > 0 else 0
    dicionario = {"engajamento_total": engajamento_total,"taxa_engajamento": round(taxa_engajamento, 2)}
    resultado = f"O engajamento total é {engajamento_total} e a taxa de engajamento é {round(taxa_engajamento, 2)}%."
    # Return the dictionary directly
    return resultado

## Vídeo 1.3 - Transformando a função em ferramenta

In [7]:
from llama_index.core.tools import FunctionTool

In [8]:
ferramenta = FunctionTool.from_defaults(fn=calcular_engajamento,name="Calcular Engajamento",description=(
        "Calcula o engajamento total e a taxa de engajamento de uma postagem nas redes sociais. "
        "Argumentos: curtidas (int), comentarios (int), compartilhamentos (int), seguidores (int). "
        "Retorna uma string com o engajamento total e a taxa de engajamento."
    ))

In [9]:
from llama_index.core.agent import FunctionCallingAgentWorker

In [10]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[ferramenta],
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm,
)

In [11]:
from llama_index.core.agent import AgentRunner

In [12]:
agent = AgentRunner(agent_worker)

In [13]:
response =agent.chat("Qual é o engajamento de uma postagem que teve 150 curtidas, 35 comentários, 20 compartilhamentos, e o perfil tem 2000 seguidores?")

Added user message to memory: Qual é o engajamento de uma postagem que teve 150 curtidas, 35 comentários, 20 compartilhamentos, e o perfil tem 2000 seguidores?
=== Calling Function ===
Calling function: Calcular Engajamento with args: {"curtidas": 150, "comentarios": 35, "compartilhamentos": 20, "seguidores": 2000}
=== Function Output ===
O engajamento total é 205 e a taxa de engajamento é 10.25%.
=== LLM Response ===
O engajamento total é 205 e a taxa de engajamento é 10.25%.


In [14]:
response = agent.chat("Quem é Albert Einstein?")

Added user message to memory: Quem é Albert Einstein?
=== LLM Response ===
Albert Einstein foi um físico teórico alemão, nascido em 14 de março de 1879, em Munique, Alemanha, e falecido em 18 de abril de 1955, em Princeton, Nova Jersey, EUA. Ele é amplamente considerado um dos cientistas mais influentes do século XX.

Einstein é mais conhecido por sua teoria da relatividade, que revolucionou a nossa compreensão do espaço e do tempo. Sua famosa equação E=mc², que relaciona a energia (E) com a massa (m) e a velocidade da luz (c), é uma das mais conhecidas e importantes equações da física.

Além disso, Einstein fez contribuições significativas em outras áreas, como a mecânica quântica, a teoria dos campos e a cosmologia. Ele também foi um defensor da paz, da liberdade e da justiça social, e foi um crítico da guerra e da opressão.

Einstein recebeu o Prêmio Nobel de Física em 1921 por sua explicação do efeito fotoelétrico, que ajudou a estabelecer a teoria quântica. Ele também foi eleito p

## Vídeo 1.4 - Consultando artigos

In [ ]:
import arxiv

In [ ]:
def consulta_artigos(titulo: str) -> str:
    """
    Consulta os artigos na base de dados arXiv e retorna resultados formatados.
    """
    busca = arxiv.Search(
        query=titulo,
        max_results=5,
        sort_by=arxiv.SortCriterion.Relevance
    )

    resultados = []
    for resultado in busca.results():
        resultados.append(f"Título: {resultado.title}\n"
                          f"Resumo: {resultado.summary}\n"
                          f"Categoria: {resultado.primary_category}\n"
                          f"Link: {resultado.entry_id}\n")

    return "\n\n".join(resultados)

In [ ]:
consulta_artigos_tool = FunctionTool.from_defaults(fn=consulta_artigos)

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [ferramenta, consulta_artigos_tool],
    verbose=True,
    allow_parallel_tool_calls=False,
)

In [ ]:
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.chat("Me retorne artigos sobre o uso da inteligência artificial nas redes sociais")

# Aula 2 - Aprofundando nas pesquisas

## Vídeo 2.1 - Adotando ferramentas prontas

In [ ]:
tavily_key = userdata.get('tavily')

In [ ]:
from llama_index.tools.tavily_research import TavilyToolSpec

tavily_tool = TavilyToolSpec(
    api_key=tavily_key,
)

In [ ]:
tavily_tool_list = tavily_tool.to_tool_list()
for tool in tavily_tool_list:
    print(tool.metadata.name)

In [ ]:
tavily_tool.search("Me retorne artigos científicos sobre o uso da inteligência artificial nas redes sociais que você encontrar na web", max_results=3)

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(tavily_tool.to_tool_list())

In [ ]:
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.chat("Me retorne artigos científicos sobre o uso da inteligência artificial nas redes sociais que você encontrar na web")

In [ ]:
print(response)

## Vídeo 2.2 - Criando uma base de dados vetorial

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

In [ ]:
url = '/content/drive/MyDrive/Alura/Cursos/4169 - LlamaIndex: Criando Agentes para utilização de ferramentas personalizadas/Projeto/artigo1.pdf'
artigo = SimpleDirectoryReader(input_files=[url]).load_data()

In [ ]:
url = '/content/drive/MyDrive/Alura/Cursos/4169 - LlamaIndex: Criando Agentes para utilização de ferramentas personalizadas/Projeto/livro1.pdf'
livro = SimpleDirectoryReader(input_files=[url]).load_data()

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [ ]:
Settings.embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-large")

In [ ]:
artigo_index = VectorStoreIndex.from_documents(artigo)
livro_index = VectorStoreIndex.from_documents(livro)

In [ ]:
artigo_index.storage_context.persist(persist_dir="/content/drive/MyDrive/Alura/Cursos/4169 - LlamaIndex: Criando Agentes para utilização de ferramentas personalizadas/Projeto/artigo")
livro_index.storage_context.persist(persist_dir="/content/drive/MyDrive/Alura/Cursos/4169 - LlamaIndex: Criando Agentes para utilização de ferramentas personalizadas/Projeto/livro")

## Vídeo 2.3 - Criando engines de busca

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage

In [ ]:
storage_context = StorageContext.from_defaults(
        persist_dir="/content/drive/MyDrive/Alura/Cursos/4169 - LlamaIndex: Criando Agentes para utilização de ferramentas personalizadas/Projeto/artigo"
    )
artigo_index = load_index_from_storage(storage_context)

storage_context = StorageContext.from_defaults(
        persist_dir="/content/drive/MyDrive/Alura/Cursos/4169 - LlamaIndex: Criando Agentes para utilização de ferramentas personalizadas/Projeto/livro"
    )
livro_index = load_index_from_storage(storage_context)

In [ ]:
artigo_engine = artigo_index.as_query_engine(similarity_top_k=3)
livro_engine = livro_index.as_query_engine(similarity_top_k=3)

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=artigo_engine,
        metadata=ToolMetadata(
            name="artigo_engine",
            description=(
                "Fornece informações sobre algoritmos de inteligência artificial nas redes sociais. "
                "Use uma pergunta detalhada em texto simples como entrada para a ferramenta."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=livro_engine,
        metadata=ToolMetadata(
            name="livro_engine",
            description=(
                "Fornece informações sobre avanços e tendências sobre inteligência artificial. "
                "Use uma pergunta detalhada em texto simples como entrada para a ferramenta."
            ),
        ),
    ),
]

# Aula 3 - Verificando o funcionamento do agente

## Vídeo 3.1 - Consultando os textos com agentes

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    verbose=True,
    allow_parallel_tool_calls=True,
)
agent_document = AgentRunner(agent_worker)

In [ ]:
response = agent_document.chat(
    "Quais os principais algoritmos de IA usados nas redes sociais?"
)

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    verbose=True,
    allow_parallel_tool_calls=True,
)
agent_document = AgentRunner(agent_worker)

In [ ]:
response = agent_document.chat(
    "Quais as principais tendências de IA que eu deveria estudar?"
)

## Vídeo 3.2 - Usando um agente ReAct

In [ ]:
from llama_index.core.agent import ReActAgent

In [ ]:
agente = ReActAgent.from_tools(
    query_engine_tools,
    verbose=True,
)

In [ ]:
response = agente.chat("Quais os principais algoritmos de IA usados nas redes sociais?")

In [ ]:
agente = ReActAgent.from_tools(
    query_engine_tools,
    verbose=True,
)

In [ ]:
response = agente.chat("Quais as principais tendências de IA que eu deveria estudar?")

In [ ]:
agente = ReActAgent.from_tools(
    query_engine_tools,
    verbose=True,
)

In [ ]:
response = agente.chat("Como o feed de uma rede social utiliza IA?")

## Vídeo 3.3 - Configurando o CrewAI

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import LlamaIndexTool
import arxiv

In [ ]:
ferramenta_artigos = FunctionTool.from_defaults(fn=consulta_artigos, name="consulta_artigos", description="Consulta os artigos na base de dados arXiv e retorna resultados formatados.")
tool = LlamaIndexTool.from_tool(ferramenta_artigos)

In [ ]:
nvidia = userdata.get('nvidia-llama405b')

In [ ]:
llm = LLM(
    model="nvidia_nim/meta/llama-3.3-70b-instruct",
    api_key=nvidia
)

In [ ]:
agent = Agent(
    role='Agente de pesquisa',
    goal='Fornece artigos científicos sobre um assunto de interesse.',
    backstory='Um agente expert em pesquisa científica que possui a habilidade de buscar artigos no arxiv',
    tools=[tool],
    llm=llm
)

In [ ]:
task1 = Task(
    description="Busque artigos científicos sobre o uso da inteligência artificial nas redes sociais. ",
    expected_output="5 artigos e seus respectivos links",
    agent=agent
)

In [ ]:
crew = Crew(
    agents=[agent],
    tasks=[task1],
    verbose=1,
)

In [ ]:
result = crew.kickoff()

print("######################")
print(result)